In [ ]:
import os
import sys
from time import time
from PIL import Image
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import networkx as nx
import random

from keras import layers, models, optimizers
from keras import backend as K
K.set_image_data_format('channels_last')
from keras.utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator
from keras import callbacks
import copy

from utils import combine_images
from utils import plot_log

from capsulelayers import CapsuleLayer, PrimaryCap, Length, Mask
sys.path.append('./PatchyTools/')

from PatchyConverter import PatchyConverter
from ConvNetPatchy import ConvNetPatchy
from DropboxLoader import DropboxLoader
from CapsuleParameters import CapsuleParameters
from CapsuleParameters import CapsuleTrainingParameters
from GraphClassifier import GraphClassifier


import matplotlib.pyplot as plt
%matplotlib inline

### Load data

In [2]:
# Getting the data:
dataset_name = 'MUTAG'
width = 18
receptive_field = 10
stride = 1
relabeling = True

In [5]:
graph_converter = PatchyConverter(dataset_name, receptive_field, stride)

importing graph data
number of graphs in MUTAG dataset : 188
number of features : 7
Separating Graphs per graph ID
getting width
width: 18


In [6]:
graph_tensor = graph_converter.graphs_to_patchy_tensor()

MUTAG graph tensor non exisiting at : /Users/marcelogutierrez/.gamma_link/Samples/MUTAG/MUTAG_patchy_bc_tensor_w_18.npy
Create dictionary of graphs
Canonical Labeling
Getting the Neighboors 
Neighboors to Tensor
MUTAG graph tensor created at: /Users/marcelogutierrez/.gamma_link/Samples/MUTAG/MUTAG_patchy_bc_tensor_w_18.npy


In [ ]:
graph_tensor = graph_converter.graph_to_patchy_tensor(labeling_procedure_name='approx_betweeness')
#graph_tensor = graph_converter.process_data()

In [7]:
# Getting the labels:
dropbox_loader = DropboxLoader(dataset_name)
graph_labels = dropbox_loader.get_graph_label()
graph_labels = np.array(graph_labels.graph_label)


In [8]:
val_acc = []
training_time = []
n_folds = 10
epochs = 400
for j in range(n_folds):
    # Starting Conv net with Patchy
    patchy_cnn = ConvNetPatchy(graph_tensor, graph_labels)
    patchy_cnn.split_data(random_state=j)
    patchy_cnn.build_graph()
    patchy_cnn.train_model(epochs=epochs)
    print('Fold {} completed with val acc : {}'.format(j, patchy_cnn.final_val_acc))

    val_acc.append(patchy_cnn.final_val_acc)
    training_time.append(patchy_cnn.training_time)



Train on 169 samples, validate on 19 samples
Epoch 1/400
169/169 [==============================] - 1s 8ms/step - loss: 0.7444 - acc: 0.3846 - val_loss: 0.6497 - val_acc: 0.6842
Epoch 2/400
169/169 [==============================] - 0s 217us/step - loss: 0.6655 - acc: 0.6036 - val_loss: 0.6022 - val_acc: 0.6842
Epoch 3/400
169/169 [==============================] - 0s 231us/step - loss: 0.5985 - acc: 0.6450 - val_loss: 0.5797 - val_acc: 0.6842
Epoch 4/400
169/169 [==============================] - 0s 215us/step - loss: 0.5750 - acc: 0.6864 - val_loss: 0.5629 - val_acc: 0.6842
Epoch 5/400
169/169 [==============================] - 0s 208us/step - loss: 0.5551 - acc: 0.6923 - val_loss: 0.5334 - val_acc: 0.6842
Epoch 6/400
169/169 [==============================] - 0s 204us/step - loss: 0.5082 - acc: 0.7101 - val_loss: 0.5067 - val_acc: 0.7895
Epoch 7/400
169/169 [==============================] - 0s 218us/step - loss: 0.4956 - acc: 0.7751 - val_loss: 0.4805 - val_acc: 0.8947
Epoch 8/400


Epoch 61/400
169/169 [==============================] - ETA: 0s - loss: 0.0252 - acc: 1.000 - 0s 172us/step - loss: 0.0197 - acc: 1.0000 - val_loss: 0.5064 - val_acc: 0.8421
Epoch 62/400
169/169 [==============================] - 0s 192us/step - loss: 0.0183 - acc: 1.0000 - val_loss: 0.5171 - val_acc: 0.8421
Epoch 63/400
169/169 [==============================] - 0s 199us/step - loss: 0.0111 - acc: 1.0000 - val_loss: 0.5367 - val_acc: 0.8421
Epoch 64/400
169/169 [==============================] - 0s 206us/step - loss: 0.0165 - acc: 1.0000 - val_loss: 0.5397 - val_acc: 0.8421
Epoch 65/400
169/169 [==============================] - 0s 189us/step - loss: 0.0148 - acc: 1.0000 - val_loss: 0.5271 - val_acc: 0.8421
Epoch 66/400
169/169 [==============================] - 0s 200us/step - loss: 0.0149 - acc: 1.0000 - val_loss: 0.5336 - val_acc: 0.8421
Epoch 67/400
169/169 [==============================] - 0s 202us/step - loss: 0.0130 - acc: 1.0000 - val_loss: 0.5526 - val_acc: 0.8421
Epoch 68/4

169/169 [==============================] - 0s 197us/step - loss: 0.0021 - acc: 1.0000 - val_loss: 0.7362 - val_acc: 0.8421
Epoch 121/400
169/169 [==============================] - 0s 207us/step - loss: 0.0023 - acc: 1.0000 - val_loss: 0.7444 - val_acc: 0.8421
Epoch 122/400
169/169 [==============================] - 0s 215us/step - loss: 0.0018 - acc: 1.0000 - val_loss: 0.7507 - val_acc: 0.8421
Epoch 123/400
169/169 [==============================] - 0s 224us/step - loss: 0.0018 - acc: 1.0000 - val_loss: 0.7562 - val_acc: 0.8421
Epoch 124/400
169/169 [==============================] - 0s 211us/step - loss: 0.0015 - acc: 1.0000 - val_loss: 0.7659 - val_acc: 0.8421
Epoch 125/400
169/169 [==============================] - 0s 225us/step - loss: 0.0023 - acc: 1.0000 - val_loss: 0.7644 - val_acc: 0.8421
Epoch 126/400
169/169 [==============================] - 0s 218us/step - loss: 0.0010 - acc: 1.0000 - val_loss: 0.7649 - val_acc: 0.8421
Epoch 127/400
169/169 [==============================] 

169/169 [==============================] - 0s 204us/step - loss: 0.0010 - acc: 1.0000 - val_loss: 0.8788 - val_acc: 0.8421
Epoch 180/400
169/169 [==============================] - 0s 181us/step - loss: 7.7380e-04 - acc: 1.0000 - val_loss: 0.8730 - val_acc: 0.8421
Epoch 181/400
169/169 [==============================] - 0s 187us/step - loss: 7.8202e-04 - acc: 1.0000 - val_loss: 0.8699 - val_acc: 0.8421
Epoch 182/400
169/169 [==============================] - 0s 196us/step - loss: 0.0015 - acc: 1.0000 - val_loss: 0.8686 - val_acc: 0.8421
Epoch 183/400
169/169 [==============================] - 0s 204us/step - loss: 5.3653e-04 - acc: 1.0000 - val_loss: 0.8686 - val_acc: 0.8421
Epoch 184/400
169/169 [==============================] - 0s 199us/step - loss: 8.0810e-04 - acc: 1.0000 - val_loss: 0.8659 - val_acc: 0.8421
Epoch 185/400
169/169 [==============================] - 0s 204us/step - loss: 6.5980e-04 - acc: 1.0000 - val_loss: 0.8677 - val_acc: 0.8421
Epoch 186/400
169/169 [============

Epoch 237/400
169/169 [==============================] - 0s 186us/step - loss: 4.7916e-04 - acc: 1.0000 - val_loss: 0.9418 - val_acc: 0.8421
Epoch 238/400
169/169 [==============================] - 0s 203us/step - loss: 2.1459e-04 - acc: 1.0000 - val_loss: 0.9414 - val_acc: 0.8421
Epoch 239/400
169/169 [==============================] - 0s 208us/step - loss: 1.9069e-04 - acc: 1.0000 - val_loss: 0.9402 - val_acc: 0.8421
Epoch 240/400
169/169 [==============================] - 0s 222us/step - loss: 1.8092e-04 - acc: 1.0000 - val_loss: 0.9387 - val_acc: 0.8421
Epoch 241/400
169/169 [==============================] - 0s 208us/step - loss: 2.7481e-04 - acc: 1.0000 - val_loss: 0.9378 - val_acc: 0.8421
Epoch 242/400
169/169 [==============================] - 0s 203us/step - loss: 2.7782e-04 - acc: 1.0000 - val_loss: 0.9387 - val_acc: 0.8421
Epoch 243/400
169/169 [==============================] - 0s 204us/step - loss: 3.7988e-04 - acc: 1.0000 - val_loss: 0.9382 - val_acc: 0.8421
Epoch 244/400

...

169/169 [==============================] - 0s 161us/step - loss: 0.0020 - acc: 1.0000 - val_loss: 0.5140 - val_acc: 0.8421
Epoch 137/400
169/169 [==============================] - 0s 161us/step - loss: 0.0019 - acc: 1.0000 - val_loss: 0.5118 - val_acc: 0.8421
Epoch 138/400
169/169 [==============================] - 0s 164us/step - loss: 0.0015 - acc: 1.0000 - val_loss: 0.5101 - val_acc: 0.8421
Epoch 139/400
169/169 [==============================] - 0s 162us/step - loss: 0.0017 - acc: 1.0000 - val_loss: 0.5093 - val_acc: 0.8421
Epoch 140/400
169/169 [==============================] - 0s 162us/step - loss: 0.0011 - acc: 1.0000 - val_loss: 0.5087 - val_acc: 0.8421
Epoch 141/400
169/169 [==============================] - 0s 162us/step - loss: 0.0016 - acc: 1.0000 - val_loss: 0.5101 - val_acc: 0.8421
Epoch 142/400
169/169 [==============================] - 0s 166us/step - loss: 0.0013 - acc: 1.0000 - val_loss: 0.5121 - val_acc: 0.8421
Epoch 143/400
169/169 [==============================] 

Epoch 194/400
169/169 [==============================] - ETA: 0s - loss: 4.2921e-04 - acc: 1.000 - 0s 166us/step - loss: 4.5764e-04 - acc: 1.0000 - val_loss: 0.5719 - val_acc: 0.8421
Epoch 195/400
169/169 [==============================] - 0s 160us/step - loss: 0.0011 - acc: 1.0000 - val_loss: 0.5711 - val_acc: 0.8421
Epoch 196/400
169/169 [==============================] - 0s 161us/step - loss: 5.2858e-04 - acc: 1.0000 - val_loss: 0.5711 - val_acc: 0.8421
Epoch 197/400
169/169 [==============================] - 0s 161us/step - loss: 0.0012 - acc: 1.0000 - val_loss: 0.5697 - val_acc: 0.8421
Epoch 198/400
169/169 [==============================] - 0s 162us/step - loss: 6.9976e-04 - acc: 1.0000 - val_loss: 0.5698 - val_acc: 0.8421
Epoch 199/400
169/169 [==============================] - 0s 159us/step - loss: 5.3546e-04 - acc: 1.0000 - val_loss: 0.5705 - val_acc: 0.8421
Epoch 200/400
169/169 [==============================] - 0s 161us/step - loss: 4.4502e-04 - acc: 1.0000 - val_loss: 0.57

169/169 [==============================] - 0s 161us/step - loss: 3.0182e-04 - acc: 1.0000 - val_loss: 0.5777 - val_acc: 0.8947
Epoch 252/400
169/169 [==============================] - 0s 157us/step - loss: 7.4672e-04 - acc: 1.0000 - val_loss: 0.5793 - val_acc: 0.8947
Epoch 253/400
169/169 [==============================] - 0s 158us/step - loss: 4.0999e-04 - acc: 1.0000 - val_loss: 0.5812 - val_acc: 0.8947
Epoch 254/400
169/169 [==============================] - 0s 160us/step - loss: 4.7810e-04 - acc: 1.0000 - val_loss: 0.5830 - val_acc: 0.8947
Epoch 255/400
169/169 [==============================] - 0s 160us/step - loss: 3.7791e-04 - acc: 1.0000 - val_loss: 0.5859 - val_acc: 0.8947
Epoch 256/400
169/169 [==============================] - 0s 155us/step - loss: 6.4658e-04 - acc: 1.0000 - val_loss: 0.5906 - val_acc: 0.8947
Epoch 257/400
169/169 [==============================] - 0s 159us/step - loss: 4.9287e-04 - acc: 1.0000 - val_loss: 0.5975 - val_acc: 0.8421
Epoch 258/400
169/169 [====

169/169 [==============================] - 0s 164us/step - loss: 1.9561e-04 - acc: 1.0000 - val_loss: 0.6259 - val_acc: 0.8947
Epoch 309/400
169/169 [==============================] - ETA: 0s - loss: 1.5777e-04 - acc: 1.000 - 0s 158us/step - loss: 1.6665e-04 - acc: 1.0000 - val_loss: 0.6252 - val_acc: 0.8947
Epoch 310/400
169/169 [==============================] - 0s 160us/step - loss: 3.3667e-04 - acc: 1.0000 - val_loss: 0.6265 - val_acc: 0.8947
Epoch 311/400
169/169 [==============================] - 0s 161us/step - loss: 2.9052e-04 - acc: 1.0000 - val_loss: 0.6296 - val_acc: 0.8947
Epoch 312/400
169/169 [==============================] - 0s 163us/step - loss: 1.5053e-04 - acc: 1.0000 - val_loss: 0.6326 - val_acc: 0.8947
Epoch 313/400
169/169 [==============================] - 0s 162us/step - loss: 3.4425e-04 - acc: 1.0000 - val_loss: 0.6351 - val_acc: 0.8947
Epoch 314/400
169/169 [==============================] - 0s 167us/step - loss: 2.8716e-04 - acc: 1.0000 - val_loss: 0.6379 - v

169/169 [==============================] - 0s 161us/step - loss: 2.0174e-04 - acc: 1.0000 - val_loss: 0.6704 - val_acc: 0.8947
Epoch 366/400
169/169 [==============================] - 0s 163us/step - loss: 1.5006e-04 - acc: 1.0000 - val_loss: 0.6723 - val_acc: 0.8947
Epoch 367/400
169/169 [==============================] - 0s 163us/step - loss: 1.3253e-04 - acc: 1.0000 - val_loss: 0.6741 - val_acc: 0.8947
Epoch 368/400
169/169 [==============================] - 0s 157us/step - loss: 8.6289e-05 - acc: 1.0000 - val_loss: 0.6761 - val_acc: 0.8947
Epoch 369/400
169/169 [==============================] - 0s 160us/step - loss: 3.2381e-04 - acc: 1.0000 - val_loss: 0.6777 - val_acc: 0.8947
Epoch 370/400
169/169 [==============================] - 0s 160us/step - loss: 1.0747e-04 - acc: 1.0000 - val_loss: 0.6789 - val_acc: 0.8947
Epoch 371/400
169/169 [==============================] - 0s 160us/step - loss: 3.8208e-04 - acc: 1.0000 - val_loss: 0.6798 - val_acc: 0.8947
Epoch 372/400
169/169 [====

In [9]:
print(np.mean(val_acc))
print(np.std(val_acc))

0.8473684251308441
0.08305123007057477
